## TYTANチュートリアル（シフト最適化）

2023年5月31日

ビネクラ安田

出典：[量子アニーリング（QUBO）でシフト最適化](https://vigne-cla.com/21-26/)

### 問題

文化祭のお店のシフトを8人の生徒から決める。

お店に必要なパワーがこちら。

|＊＊＊＊＊＊|13時|14時|15時|16時|17時|
|---|:---:|:---:|:---:|:---:|:---:|
|必要なパワー|2|2|2|2|2|

8人の生徒のシフト希望がこちら。経験者はパワー2を持つ。

|＊＊＊＊＊＊|13時|14時|15時|16時|17時|
|---|:---:|:---:|:---:|:---:|:---:|
|q0|1||||1|
|q1|1|||1||
|q2||1|||1|
|q3|||1|1||
|q4||||1|1|
|q5（経験者）|2|||||
|q6（経験者）||2||||
|q7（経験者）|||2|||

生徒は、採用であればきっちりシフト希望通り入ること。q0君が採用された場合、一部だけ入るようなことはできず必ず13時・17時の両方に入る。


### QUBOモデルでは何が設定できるか？（おさらい）

応用D1（解が0と1だけの方程式）を使う。

**重さ1, 2, 3のボールのどれを取れば合計3の重さになるか？**のは次のように設定できる。
```
H = (1*q0 + 2*q1 + 3*q2 - 3)**2
```

解は [q0, q1, q2] = [0, 0, 1] or [1, 1, 0] となり、1, 2のボール、または3のボールを取れば良いという結果になる。

その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 制約条件

すでに表中にq0と書いているように、まずは生徒に量子ビットを割り当てて、１になったら勤務、０になったらお休みとする。

条件式の設定は意外とシンプルで、13時の列に着目すると<font color="red">「q0, q1, q5 の合計が2になる」</font>必要がある。q5は経験者なので重み2であることに注意すると、

```
#13時の列
H = (q0 + q1 + 2*q5 - 2)**2
```
となり、これを他の時間帯でも同様に設定するだけで良い。

## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [ ]:
from tytan import symbols, Compile, sampler

#量子ビットを用意する
q0 = symbols('q0')
q1 = symbols('q1')
q2 = symbols('q2')
q3 = symbols('q3')
q4 = symbols('q4')
q5 = symbols('q5')
q6 = symbols('q6')
q7 = symbols('q7')

#各時間帯とも、パワー合計が2になる
H = 0
H += (q0 + q1 + 2*q5 - 2)**2
H += (q2 + 2*q6 - 2)**2
H += (q3 + 2*q7 - 2)**2
H += (q1 + q3 + q4 - 2)**2
H += (q0 + q2 + q4 - 2)**2


#コンパイル
qubo, offset = Compile(H).get_qubo()

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=500)

#結果
for r in result[:5]:
    print(r)

[{'q0': 1.0, 'q1': 1.0, 'q2': 0.0, 'q3': 0.0, 'q4': 1.0, 'q5': 0.0, 'q6': 1.0, 'q7': 1.0}, -20.0, 257]
[{'q0': 0.0, 'q1': 0.0, 'q2': 0.0, 'q3': 1.0, 'q4': 1.0, 'q5': 1.0, 'q6': 1.0, 'q7': 0.0}, -18.0, 14]
[{'q0': 0.0, 'q1': 0.0, 'q2': 0.0, 'q3': 1.0, 'q4': 1.0, 'q5': 1.0, 'q6': 1.0, 'q7': 1.0}, -18.0, 21]
[{'q0': 0.0, 'q1': 0.0, 'q2': 1.0, 'q3': 0.0, 'q4': 1.0, 'q5': 1.0, 'q6': 0.0, 'q7': 1.0}, -18.0, 21]
[{'q0': 0.0, 'q1': 0.0, 'q2': 1.0, 'q3': 0.0, 'q4': 1.0, 'q5': 1.0, 'q6': 1.0, 'q7': 1.0}, -18.0, 9]


解のエネルギーがマイナスoffsetと同じ値であることは、すべての条件を満たしたシフトであることを意味する。

採用した生徒を確認すると、どの時間帯もパワー2になっている。

|＊＊＊＊＊＊|13時|14時|15時|16時|17時|
|---|:---:|:---:|:---:|:---:|:---:|
|q0|1||||1|
|q1|1|||1||
|q4||||1|1|
|q6（経験者）||2||||
|q7（経験者）|||2|||

### おまけ１
お店に必要なパワーが時間帯によって違っても考え方はそのまま。ピーク帯とかあるので。

また、問題によっては他の制約条件も足せる。

例えば、「全部で3人採用する」とあれば「8つの量子ビットから3つを1にする」
```
#8人から3人を採用
H = (q0 + q1 + q2 + q3 + q4 + q5 + q6 + q7 - 3)**2
```

例えば、「q0君とq1君は仲が悪いので同時には採用しない」とあれば「2つの量子ビットが同時に1になったらペナルティ」
```
#q0とq1は仲が悪い
H = (q0 *q1)
```
などを追加する。条件の強さに応じて重みを付ける必要もあり得る。

### おまけ２

for文で式を設定する場合は、「可否＊パワー＊量子ビット」を並べると良いだろう。可否が0の項は無視されるので意味は同じである。（シグマの数式で固く説明されているのはこのような形）

```
#13時の列
H += (q0 + q1 + 2*q5 - 2)**2
```
↓
```
H += (1*1*q0 + 1*1*q1 + 0*1*q2 + 0*1*q3 + 0*1*q4 + 1*2*q5 + 0*2*q6 + 0*2*q7 - 2)**2
```